# ETL Project
#### New Film Project
Our group has decided to make a new film. But before we commit to a new project, we will gather data on past succesful films. We will look at starting budgets, gross revenue, genres and other relevant information displayed below in our panda data frames and postgres tables later on. 
### Extract
All our data was found on website [Kaggle](https://www.Kaggle.com) in CSV format. 
These are the direct links:
[Blockbuster CSV ](https://www.kaggle.com/bidyutchanda/top-10-highest-grossing-films-19752018) <br><br>
[IMDB CSV](https://www.kaggle.com/PromptCloudHQ/imdb-data)<br><br>
[TMBD CSV](https://www.kaggle.com/tmdb/tmdb-movie-metadata)
### Transform
After downloading our, data we used a Jupyter Notebook to import the CSVs and created data frames using the Pandas library.
<br>
<br>
Once they were Panda data frames form we did the following to clean and transform our data to make it useful:
<br>
* Drop columns we found irrelevant or were duplicates <br>
* Renamed columns to make it look cleaner and match columns on postgres <br>
* We dropped any possible duplicate records from our tables <br>
* Set an index for each of our data frames <br>
* Changed data type on certain columns to be able able to import onto postgres database tables <br>
### Load
After cleaning our data, we imported all data frames onto postres database tables. Once on postress, we were able to run queries to find useful information like:
<br>
* Top 10 highest earning films <br>
* Genre with the highest earnings <br>
* Directors who made top films <br>
* Return on films <br>
* Studios with the higest revenue <br>
<br>
<br>
Why we chose postres? <br>
<br>
We chose postgres because it is a platform we are more familiar with in regard to manipulating tables (joining, primary keys, etc.)

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
blockbuster_file = 'Resources/blockbusters.csv'
blockbuster_df = pd.read_csv(blockbuster_file)
new_blockbuster_df = blockbuster_df[['title',
                                     'Main_Genre',
                                     'rating',
                                     'year',
                                     'worldwide_gross',
                                    'studio']]
new_blockbuster_df = new_blockbuster_df.rename(columns={'title': 'title',
                                  'Main_Genre': 'genre',
                                  'rating': 'rating',
                                  'year': 'release_year',
                                  'worldwide_gross': 'worldwide_gross',
                                                       'studio': 'studio'})
new_blockbuster_df.drop_duplicates('title', inplace=True)
new_blockbuster_df.set_index('title', inplace=True)
new_blockbuster_df.head()

,genre,rating,release_year,worldwide_gross,studio
title,,,,,
Black Panther,Action,PG-13,2018,"$700,059,566",Walt Disney Pictures
Avengers: Infinity War,Action,PG-13,2018,"$678,815,482",Walt Disney Pictures
Incredibles 2,Animation,PG,2018,"$608,581,744",Pixar
Jurassic World: Fallen Kingdom,Action,PG-13,2018,"$416,769,345",Universal Pictures
Deadpool 2,Action,R,2018,"$318,491,426",20th Century Fox


In [3]:
new_blockbuster_df['worldwide_gross'] = (new_blockbuster_df['worldwide_gross']
                                         .replace( '[\$,)]','', regex=True )
                                         .replace( '[(]','-',   regex=True ).astype(float))



In [4]:
imdb_file = "Resources/IMDB-Movie-Data.csv"
imdb_df = pd.read_csv(imdb_file)
new_imdb_df = imdb_df[['Title', 'Director', 'Actors', 'Rating',
                       'Votes','Revenue (Millions)','Metascore']]
new_imdb_df = new_imdb_df.rename(columns={'Title':'title', 'Director':'director',
                                     'Actors':'actors','Rating':'rating',
                                     'Votes':'votes',
                                     'Revenue (Millions)':'revenue_millions',
                                     'Metascore':'metascore'})
new_imdb_df.drop_duplicates('title', inplace = True)
new_imdb_df.set_index('title', inplace = True)
new_imdb_df.head()

,director,actors,rating,votes,revenue_millions,metascore
title,,,,,,
Guardians of the Galaxy,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",8.1,757074,333.13,76.0
Prometheus,Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",7.0,485820,126.46,65.0
Split,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",7.3,157606,138.12,62.0
Sing,Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",7.2,60545,270.32,59.0
Suicide Squad,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",6.2,393727,325.02,40.0


In [5]:
tmdb_file = "Resources/tmdb_5000_movies.csv"
tmdb_df = pd.read_csv(tmdb_file)
new_tmdb_df = tmdb_df[['original_title','budget','runtime']]
new_tmdb_df.head()
new_tmdb_df = new_tmdb_df.rename(columns={'original_title': 'title',
                                         'budget': 'budget',
                                         'runtime': 'runtime'})
new_tmdb_df.drop_duplicates('title', inplace=True)
new_tmdb_df.set_index('title', inplace=True)
new_tmdb_df.head()

,budget,runtime
title,,
Avatar,237000000,162.0
Pirates of the Caribbean: At World's End,300000000,169.0
Spectre,245000000,148.0
The Dark Knight Rises,250000000,165.0
John Carter,260000000,132.0


In [6]:
rds_connection_string = "postgres:Ct20160620@@localhost:5432/etl_movies_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [7]:
engine.table_names()

['blockbuster', 'imdb', 'tmdb']

In [8]:
new_blockbuster_df.to_sql(name='blockbuster', con=engine, if_exists='append', index=True)

In [9]:
new_imdb_df.to_sql(name='imdb', con=engine, if_exists='append', index=True)

In [10]:
new_tmdb_df.to_sql(name='tmdb', con=engine, if_exists='append', index=True)